# Contributions

Extract campaign contributions linked to a committee

In [1]:
import os
import settings
import pandas as pd

### Pull in all filings

In [2]:
filings = pd.read_csv(os.path.join(settings.data_dir, "filings.csv"))

In [3]:
filings.head()

,calaccess_committee_id,committee_name,calaccess_filing_id,date_filed,from_date,thru_date,total_contributions,total_expenditures_made
0,1340976,"2012 AUTO INSURANCE DISCOUNT ACT, YES ON PROPO...",1622725,2012-04-30,2011-01-01,2011-09-30,8227127.0,1083096.0
1,1340976,"2012 AUTO INSURANCE DISCOUNT ACT, YES ON PROPO...",1635951,2012-04-30,2011-10-01,2011-12-31,47751.0,1264650.0
2,1340976,"2012 AUTO INSURANCE DISCOUNT ACT, YES ON PROPO...",1657731,2012-06-08,2012-01-01,2012-03-31,1707.0,236438.0
3,1340976,"2012 AUTO INSURANCE DISCOUNT ACT, YES ON PROPO...",1677945,2012-07-30,2012-04-01,2012-06-30,21497.0,239163.0
4,1340976,"2012 AUTO INSURANCE DISCOUNT ACT, YES ON PROPO...",1694941,2013-01-15,2012-07-01,2012-09-30,31800.0,3875759.0


### Pull in all contributions

In [4]:
monetary_sql_template = """
SELECT *
FROM calaccess_processed_form460scheduleaitem
WHERE filing_id IN ({})
"""

In [5]:
monetary_sql = monetary_sql_template.format(
    ",".join(map(str, list(filings.calaccess_filing_id)))
)

In [6]:
contribs = pd.read_sql_query(monetary_sql, settings.db_connection)

In [7]:
trimmed_contribs = contribs.rename(columns={
    'filing_id': 'calaccess_filing_id',
    'id': 'ccdc_contrib_id',
    'transaction_id': 'calaccess_contrib_id'
}).drop([
    'line_item',
    'date_received_thru',
    'transaction_type',
    'memo_reference_number',
    'cumulative_ytd_amount',
    'cumulative_election_amount',
], axis=1)

In [8]:
trimmed_contribs.head()

,ccdc_contrib_id,date_received,calaccess_contrib_id,contributor_code,contributor_committee_id,contributor_title,contributor_lastname,contributor_firstname,contributor_name_suffix,contributor_city,...,intermediary_firstname,intermediary_name_suffix,intermediary_city,intermediary_state,intermediary_zip,intermediary_employer,intermediary_occupation,intermediary_is_self_employed,amount,calaccess_filing_id
0,210652300,2000-02-08,INC8,OTH,,,GARDEN HIGHWAY MUTUAL WATER COMPANY,,,YUBA CITY,...,,,,,,,,False,234.82,605917
1,210652301,2000-02-08,INC10,OTH,,,GROSS FARMS,J.J.,,GRIMES,...,,,,,,,,False,126.80,605917
2,210652302,2000-02-08,INC15,OTH,,,M & T CHICO RANCH,,,CHICO,...,,,,,,,,False,263.72,605917
3,210652303,2000-02-08,INC16,OTH,,,NATOMAS MUTUAL WATER COMPANY,,,RIO LINDA,...,,,,,,,,False,2563.73,605917
4,210652304,2000-02-08,INC17,OTH,,,OJI BROTHERS FARM INCORPORATED,,,YUBA CITY,...,,,,,,,,False,100.47,605917


### Merge them 

In [9]:
merged = pd.merge(
    filings,
    trimmed_contribs,
    how="inner",
    on="calaccess_filing_id"
)

In [10]:
unwanted_columns = [
    'from_date',
    'thru_date',
    'total_contributions',
    'total_expenditures_made',
    'ccdc_contrib_id',
    'date_filed',
    'calaccess_contrib_id',
    'contributor_code',
    'contributor_committee_id',
    'contributor_title',
    'contributor_name_suffix',
    'intermediary_committee_id',
    'intermediary_title',
    'intermediary_lastname',
    'intermediary_firstname',
    'intermediary_name_suffix',
    'intermediary_city',
    'intermediary_state',
    'intermediary_zip',
    'intermediary_employer',
    'intermediary_occupation',
    'intermediary_is_self_employed'
]

In [11]:
trimmed_merged = merged.drop(unwanted_columns, axis=1)

In [12]:
trimmed_merged.head()

,calaccess_committee_id,committee_name,calaccess_filing_id,date_received,contributor_lastname,contributor_firstname,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_is_self_employed,amount
0,1340976,"2012 AUTO INSURANCE DISCOUNT ACT, YES ON PROPO...",1622725,2011-08-26,JOSEPH,GEORGE,LOS ANGELES,CA,90010,MERCURY GENERAL CORPORATION,CHAIRMAN,False,8077126.97
1,1340976,"2012 AUTO INSURANCE DISCOUNT ACT, YES ON PROPO...",1635951,2011-10-20,ABERNATHY INSURANCE AGENCY INC.,,ARCADIA,CA,91006,,,False,14000.00
2,1340976,"2012 AUTO INSURANCE DISCOUNT ACT, YES ON PROPO...",1635951,2011-11-02,CALGARD ASSOCIATES,,SAN DIMAS,CA,91773,,,False,10000.00
3,1340976,"2012 AUTO INSURANCE DISCOUNT ACT, YES ON PROPO...",1635951,2011-12-05,CISNEROS,CARLOS,WILMINGTON,CA,90744,UNIAMERICA INSURANCE INC.,BROKER-AGENT,False,250.00
4,1340976,"2012 AUTO INSURANCE DISCOUNT ACT, YES ON PROPO...",1635951,2011-10-01,CONREY,JERRY,TUSTIN,CA,92780,SIATT INSURANCE CORPORATION,INSURANCE AGENCY PRINCIPAL,False,225.00


In [13]:
trimmed_merged.to_csv(
    os.path.join(settings.data_dir, 'contributions.csv'),
    index=False
)

### Filter down to 2016 contributions

In [14]:
recent_filings = pd.read_csv(os.path.join(settings.data_dir, "filings-2016.csv"))

In [15]:
recent_contribs = trimmed_merged[trimmed_merged.calaccess_filing_id.isin(recent_filings.calaccess_filing_id)]

In [16]:
recent_contribs.head(10)

,calaccess_committee_id,committee_name,calaccess_filing_id,date_received,contributor_lastname,contributor_firstname,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_is_self_employed,amount
3829,1386560,ADULT USE CAMPAIGN FOR PROPOSITION 64; THE,2083796,2016-09-18,BERGMAN,GRETCHEN,SPRING VALLEY,CA,91978,A NEW PATH,EXECUTIVE DIRECTOR,False,84.0
3830,1386560,ADULT USE CAMPAIGN FOR PROPOSITION 64; THE,2083796,2016-09-18,KAHLE,MYRNA,SAN DIEGO,CA,92109,NATIONAL SCHOOL DISTRICT,TEACHER,False,35.0
3831,1386560,ADULT USE CAMPAIGN FOR PROPOSITION 64; THE,2083796,2016-07-15,MCDEVITT,LEO,ESCONDIDO,CA,92025,LIFE IONIZERS,SEO/CONTENT MANAGER,False,198.0
3832,1386560,ADULT USE CAMPAIGN FOR PROPOSITION 64; THE,2083796,2016-08-10,WARREN-SAMARIPA,STEPHANIE,SAN DIEGO,CA,92116,STEPHANIE WARREN SAMARIPA,ENTREPRENEUR,False,-50.0
3833,1386560,ADULT USE CAMPAIGN FOR PROPOSITION 64; THE,2083796,2016-07-26,WARREN-SAMARIPA,STEPHANIE,SAN DIEGO,CA,92116,STEPHANIE WARREN SAMARIPA,ENTREPRENEUR,True,50.0
3834,1386560,ADULT USE CAMPAIGN FOR PROPOSITION 64; THE,2083796,2016-07-26,VENTURO,JOHN,SAN DIEGO,CA,92110,SMI,ENGINEER,False,198.0
3835,1386560,ADULT USE CAMPAIGN FOR PROPOSITION 64; THE,2083796,2016-07-29,HANNON,SUSAN,CARLSBAD,CA,92008,SUSAN HANNON,SALES,True,100.0
3836,1386560,ADULT USE CAMPAIGN FOR PROPOSITION 64; THE,2083796,2016-08-06,SCHERER,WAYNE,SAN DIEGO,CA,92117,WAYNE SCHERER,OWNER,True,30.0
3837,1386560,ADULT USE CAMPAIGN FOR PROPOSITION 64; THE,2083796,2016-08-19,SEXTON,MICHELLE,SAN DIEGO,CA,92109,CENTER FOR THE STUDY OF CANNABIS AND SOCIAL PO...,RESEARCHER,False,84.0
3838,1386560,ADULT USE CAMPAIGN FOR PROPOSITION 64; THE,2083796,2016-09-15,WOLFF,MICHELLE,SAN DIEGO,CA,92107,MICHELLE WOLFF,COMEDIAN,True,84.0


In [17]:
recent_contribs.to_csv(
    os.path.join(settings.data_dir, 'contributions-2016.csv'),
    index=False
)